## Initializing sigLIP

In [1]:
from transformers import pipeline
from PIL import Image
import requests

# load pipe
siglip = pipeline(task="zero-shot-image-classification", model="google/siglip-base-patch16-224")

# load image
url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

# inference
candidate_labels = ["2 cats", "a plane", "a remote"]
logits = siglip(image, candidate_labels=candidate_labels)
output = max(logits, key=lambda x: x['score'])['label']
print(output)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


2 cats


## Running this on CIFAR10

In [13]:
import torch
from transformers import ViTForImageClassification
from transformers import AutoImageProcessor
from torch.utils.data import Dataset
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision import transforms

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True)

classes = test_dataset.classes

Files already downloaded and verified


Single Image Setup

In [11]:
logits = siglip(test_dataset[0][0], candidate_labels=classes)
output = max(logits, key=lambda x: x['score'])['label']

print(f"Output: {output}, True Label: {classes[test_dataset[0][1]]}")

Output: cat, True Label: cat


Setting Up Code for Extensive Inference

In [14]:
total = 0
correct = 0
for i, image in enumerate(test_dataset):
  logits = siglip(image[0], candidate_labels=classes)
  output = max(logits, key=lambda x: x['score'])['label']
  if (output == classes[image[1]]):
    correct += 1
  total +=1
  print(f"{correct}/{total}")
  if(i==200):
    break #TESTING ON 200 IMAGES CUZ V SLOW AND ASGNMENT IS DDUEEEE


acc = 100 * correct / total

print("sigLIP Accuracy on CIFAR 10: ", acc)

1/1
2/2
3/3
3/4
4/5
5/6
6/7
7/8
8/9
9/10
10/11
11/12
12/13
13/14
14/15
15/16
16/17
17/18
18/19
19/20
20/21
21/22
22/23
23/24
24/25
25/26
26/27
27/28
28/29
29/30
30/31
31/32
32/33
33/34
34/35
35/36
35/37
35/38
36/39
37/40
38/41
39/42
39/43
40/44
41/45
42/46
42/47
42/48
43/49
44/50
45/51
46/52
47/53
48/54
49/55
50/56
51/57
52/58
53/59
53/60
54/61
55/62
56/63
57/64
58/65
59/66
60/67
61/68
62/69
63/70
64/71
65/72
66/73
67/74
67/75
68/76
69/77
70/78
71/79
72/80
73/81
74/82
75/83
76/84
77/85
78/86
79/87
80/88
81/89
82/90
83/91
84/92
85/93
86/94
87/95
88/96
89/97
90/98
91/99
92/100
93/101
94/102
95/103
96/104
97/105
98/106
99/107
100/108
101/109
102/110
103/111
104/112
105/113
106/114
107/115
108/116
109/117
110/118
111/119
112/120
113/121
114/122
115/123
116/124
117/125
118/126
119/127
120/128
120/129
121/130
122/131
123/132
124/133
125/134
126/135
127/136
128/137
129/138
130/139
131/140
132/141
133/142
134/143
135/144
136/145
137/146
138/147
139/148
140/149
141/150
142/151
143/152
144/153
1

## RUNNING ON SVHN
Loading SVHN

In [2]:
!pip install datasets

In [3]:
from datasets import load_dataset

ds = load_dataset("ufldl-stanford/svhn", "cropped_digits")

classes = ds['train'].features['label'].names

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/136M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/47.0M [00:00<?, ?B/s]

extra-00000-of-00002.parquet:   0%|          | 0.00/511M [00:00<?, ?B/s]

extra-00001-of-00002.parquet:   0%|          | 0.00/512M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/73257 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/26032 [00:00<?, ? examples/s]

Generating extra split:   0%|          | 0/531131 [00:00<?, ? examples/s]

Inference on 200 Images from SVHN

In [7]:
total = 0
correct = 0
for i, image in enumerate(ds["test"]):
  logits = siglip(image['image'], candidate_labels=classes)
  output = max(logits, key=lambda x: x['score'])['label']
  if (output == classes[image['label']]):
    correct += 1
  total +=1

  print(f"Output: {output}, True: {classes[image['label']]} : {correct}/{total}")
  if(i==200):
    break #TESTING ON 200 IMAGES CUZ V SLOW AND ASGNMENT IS DDUEEEE


acc = 100 * correct / total

print("sigLIP Accuracy on CIFAR 10: ", acc)

Output: 7, True: 5 : 0/1
Output: 0, True: 2 : 0/2
Output: 1, True: 1 : 1/3
Output: 0, True: 0 : 2/4
Output: 0, True: 6 : 2/5
Output: 0, True: 1 : 2/6
Output: 0, True: 9 : 2/7
Output: 2, True: 1 : 2/8
Output: 0, True: 1 : 2/9
Output: 0, True: 8 : 2/10
Output: 8, True: 3 : 2/11
Output: 1, True: 6 : 2/12
Output: 2, True: 5 : 2/13
Output: 0, True: 1 : 2/14
Output: 2, True: 4 : 2/15
Output: 0, True: 4 : 2/16
Output: 2, True: 1 : 2/17
Output: 0, True: 6 : 2/18
Output: 3, True: 3 : 3/19
Output: 4, True: 4 : 4/20
Output: 2, True: 2 : 5/21
Output: 0, True: 0 : 6/22
Output: 0, True: 1 : 6/23
Output: 0, True: 3 : 6/24
Output: 2, True: 2 : 7/25
Output: 5, True: 5 : 8/26
Output: 4, True: 4 : 9/27
Output: 1, True: 1 : 10/28
Output: 1, True: 4 : 10/29
Output: 1, True: 2 : 10/30
Output: 3, True: 8 : 10/31
Output: 2, True: 3 : 10/32
Output: 2, True: 8 : 10/33
Output: 6, True: 6 : 11/34
Output: 0, True: 0 : 12/35
Output: 1, True: 1 : 13/36
Output: 5, True: 5 : 14/37
Output: 0, True: 1 : 14/38
Output: 1,

## Evaluating PACS

Loading Dataset

In [9]:
from datasets import load_dataset

ds = load_dataset("flwrlabs/pacs")

classes = ds['train'].features['label'].names

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/191M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9991 [00:00<?, ? examples/s]

##  Running Inference on 200 Images

In [12]:
import random

total = 0
correct = 0
num_samples = 200  # Number of random samples you want to evaluate
indices = random.sample(range(len(ds["train"])), num_samples)  # Randomly select indices

for idx in indices:
    image = ds["train"][idx]
    logits = siglip(image['image'], candidate_labels=classes)
    output = max(logits, key=lambda x: x['score'])['label']
    if output == classes[image['label']]:
        correct += 1
    total += 1
    print(f"Output: {output}, True: {classes[image['label']]} : {correct}/{total}")

acc = 100 * correct / total
print("sigLIP Accuracy on CIFAR 10: ", acc)

Output: house, True: house : 1/1
Output: elephant, True: elephant : 2/2
Output: giraffe, True: giraffe : 3/3
Output: giraffe, True: giraffe : 4/4
Output: person, True: person : 5/5
Output: horse, True: horse : 6/6
Output: elephant, True: elephant : 7/7
Output: giraffe, True: giraffe : 8/8
Output: horse, True: horse : 9/9
Output: giraffe, True: giraffe : 10/10
Output: elephant, True: elephant : 11/11
Output: elephant, True: elephant : 12/12
Output: giraffe, True: giraffe : 13/13
Output: person, True: person : 14/14
Output: giraffe, True: giraffe : 15/15
Output: elephant, True: dog : 15/16
Output: house, True: house : 16/17
Output: person, True: person : 17/18
Output: person, True: person : 18/19
Output: giraffe, True: giraffe : 19/20
Output: dog, True: dog : 20/21
Output: horse, True: horse : 21/22
Output: house, True: house : 22/23
Output: giraffe, True: giraffe : 23/24
Output: person, True: person : 24/25
Output: guitar, True: guitar : 25/26
Output: house, True: person : 25/27
Output: